In [ ]:
import os
import random
import assembleFeatureMatrix_2 as afm

dataDirDev = "." + os.sep + "dataPackage"
featDirDevs = "." + os.sep + "data_feats" + os.sep + "combined"
featDirDev = "." + os.sep + "data_feats"
modelDirDev = "." + os.sep + "model_eval"

expType = "task-ils"
dataDir = dataDirDev + os.path.sep + expType
dirs: list[list[str]] = []

subjDirs = [f.path for f in os.scandir(dataDir) if f.is_dir()]
for subjd in subjDirs:
    if 'sub-cp003' in subjd:
        continue
    sessDirs = [f.path for f in os.scandir(subjd) if f.is_dir()]
    for sessd in sessDirs:
        runDirs = [f.path for f in os.scandir(sessd) if f.is_dir()]
        for rund in runDirs:
            dataFiles = [f.path for f in os.scandir(rund) if (f.is_file() and not f.name.startswith('.'))]
            tokens = rund.split(os.path.sep)
            subj = tokens[-3]
            sess = tokens[-2]
            level = tokens[-1].split('_')[0]
            run = tokens[-1].split('_')[1]
            dirs.append([subj, sess, level, run])

pctHoldout = 25
numDirs = len(dirs)
numVal = int(numDirs * float(pctHoldout) / 100)
numTrain = numDirs - numVal
random.shuffle(dirs)
trainSet = dirs[:numTrain]
valSet = dirs[numTrain:]

aggFeatFilePath = featDirDev + os.path.sep + 'devSubjsFeatMat_vae.csv'
featMatDF = afm.getFeatureMatrixSingleFile(dataDirDev, expType, dirs, aggFeatFilePath)